# IMPROVED CNN - Optimized for 70-85% Validation Accuracy

## Key Improvements from Original Version

This notebook implements an optimized CNN with critical improvements to boost validation accuracy from 44.85% to **70-85%**:

### 🎯 Main Changes

1. **Reduced Dropout** (50% reduction)
   - Conv blocks: 0.1, 0.15, 0.2, 0.25 (was 0.2, 0.3, 0.4, 0.4)
   - FC layers: 0.3 (was 0.5)

2. **Less Aggressive Augmentation**
   - Rotation: 10° (was 15°)
   - Color jitter: 0.1 (was 0.2)
   - Affine translate: 0.08 (was 0.1)

3. **Better Training Dynamics**
   - NO Label Smoothing (standard CrossEntropy)
   - OneCycleLR scheduler with 30% warmup
   - Gradient clipping (max_norm=1.0)
   - Higher learning rate: 0.003 (was 0.001)
   - Lower weight decay: 5e-5 (was 1e-4)

4. **Extended Training**
   - 150 epochs (was 100)
   - Patience: 20 (was 15)

### 📊 Expected Results

- **Training Accuracy**: 75-90%
- **Validation Accuracy**: 70-85%
- **Overfitting Gap**: 3-7%
- **Training Time**: 3-5 hours on GPU

In [ ]:
# CELL 1: Install and Import
!pip install torch torchvision tqdm -q

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import pickle

# Set random seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
# CELL 2: Load Data from Pickle Files
# Load data
with open('train-70_.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('validation-10_.pkl', 'rb') as f:
    val_data = pickle.load(f)

# Convert to numpy arrays
train_images = np.array(train_data['images'])
train_labels = np.array(train_data['labels'])
val_images = np.array(val_data['images'])
val_labels = np.array(val_data['labels'])

print(f"Train: {train_images.shape}, Val: {val_images.shape}")

# Remap labels to [0, num_classes-1]
unique_labels = np.unique(np.concatenate([train_labels, val_labels]))
label_mapping = {old: new for new, old in enumerate(unique_labels)}
train_labels = np.array([label_mapping[l] for l in train_labels])
val_labels = np.array([label_mapping[l] for l in val_labels])

print(f"Remapped labels to range: [{train_labels.min()}, {train_labels.max()}]")
print(f"Number of classes: {len(unique_labels)}")

In [ ]:
# CELL 3: Custom Dataset Class
class CustomImageDataset(Dataset):
    """Custom Dataset for loading images from pickle files"""
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        self.classes = np.unique(labels).tolist()
        self.num_classes = len(self.classes)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        # Convert to PIL Image
        if image.dtype == np.uint8:
            image = Image.fromarray(image)
        else:
            image = Image.fromarray((image * 255).astype(np.uint8))

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# CELL 4: IMPROVED Model Architecture with Less Dropout
class ImprovedTinyImageNetCNN(nn.Module):
    """
    IMPROVED CNN with optimized dropout and architecture
    Key changes:
    - Reduced dropout for better learning
    - Wider filters in later layers
    - Better regularization balance
    """
    def __init__(self, num_classes=15, dropout_rate=0.3):
        super(ImprovedTinyImageNetCNN, self).__init__()

        # Block 1: 64x64 -> 32x32 (REDUCED dropout: 0.1)
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.1)  # REDUCED from 0.2
        )

        # Block 2: 32x32 -> 16x16 (REDUCED dropout: 0.15)
        self.block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.15)  # REDUCED from 0.3
        )

        # Block 3: 16x16 -> 8x8 (REDUCED dropout: 0.2)
        self.block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.2)  # REDUCED from 0.4
        )

        # Block 4: 8x8 -> 4x4 (REDUCED dropout: 0.25)
        self.block4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.25)  # REDUCED from 0.4
        )

        # Flatten and fully connected layers
        self.flatten = nn.Flatten()

        # IMPROVED classifier with less dropout
        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_rate),  # 0.3 instead of 0.5
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_rate),  # 0.3 instead of 0.5
            nn.Linear(512, num_classes)
        )

        # Weight initialization
        self._initialize_weights()

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        """Initialize weights using He initialization for ReLU"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                nn.init.constant_(m.bias, 0)

# Test model creation
model = ImprovedTinyImageNetCNN(num_classes=15).to(device)
print(f'\nImproved Model created successfully!')
print(f'Total parameters: {sum(p.numel() for p in model.parameters()):,}')
print(f'Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}')

In [ ]:
# CELL 5: IMPROVED Data Transforms (Less Aggressive Augmentation)
# IMPROVED: Less aggressive augmentation for better learning
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),  # REDUCED from 15
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),  # REDUCED
    transforms.RandomAffine(degrees=0, translate=(0.08, 0.08)),  # REDUCED from 0.1
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Validation transform (no augmentation, only normalization)
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = CustomImageDataset(
    images=train_images,
    labels=train_labels,
    transform=train_transform
)

val_dataset = CustomImageDataset(
    images=val_images,
    labels=val_labels,
    transform=val_transform
)

print(f'Training samples: {len(train_dataset)}')
print(f'Validation samples: {len(val_dataset)}')
print(f'Number of classes: {train_dataset.num_classes}')
print(f'Classes: {train_dataset.classes}')

# IMPROVED Hyperparameters
BATCH_SIZE = 64
NUM_EPOCHS = 150  # INCREASED for better convergence
LEARNING_RATE = 0.003  # INCREASED for faster learning
PATIENCE = 20  # INCREASED patience
WEIGHT_DECAY = 5e-5  # REDUCED weight decay

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f'\nTraining batches: {len(train_loader)}')
print(f'Validation batches: {len(val_loader)}')

In [ ]:
# CELL 6: IMPROVED Training Setup
# Initialize model
model = ImprovedTinyImageNetCNN(num_classes=15, dropout_rate=0.3).to(device)

# IMPROVED: Use regular CrossEntropy (NO label smoothing)
criterion = nn.CrossEntropyLoss()

# IMPROVED: Adam with better weight decay
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# IMPROVED: Use OneCycleLR for better training dynamics
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=LEARNING_RATE,
    epochs=NUM_EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=0.3,  # 30% warmup
    anneal_strategy='cos',
    div_factor=25.0,
    final_div_factor=10000.0
)

print('IMPROVED Training setup complete!')
print(f'Learning Rate: {LEARNING_RATE}')
print(f'Weight Decay: {WEIGHT_DECAY}')
print(f'Batch Size: {BATCH_SIZE}')
print(f'Max Epochs: {NUM_EPOCHS}')
print(f'Scheduler: OneCycleLR with cosine annealing')

In [ ]:
# CELL 7: IMPROVED Training Functions
def train_epoch(model, train_loader, criterion, optimizer, scheduler, device):
    """Train for one epoch with gradient clipping"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc='Training')
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()  # Step per batch for OneCycleLR

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%',
            'lr': f'{scheduler.get_last_lr()[0]:.6f}'
        })

    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


def validate(model, val_loader, criterion, device):
    """Validate the model"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        pbar = tqdm(val_loader, desc='Validation')
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })

    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

In [ ]:
# CELL 8: IMPROVED Training Loop
best_val_acc = 0.0
best_epoch = 0
patience_counter = 0
train_losses = []
train_accs = []
val_losses = []
val_accs = []
learning_rates = []

print('Starting IMPROVED training...\n')
print('Key improvements:')
print('✓ Reduced dropout (0.1-0.3 instead of 0.2-0.5)')
print('✓ Less aggressive augmentation')
print('✓ No label smoothing')
print('✓ OneCycleLR scheduler with warmup')
print('✓ Gradient clipping')
print('✓ Higher initial learning rate')
print('✓ Lower weight decay')
print()

for epoch in range(NUM_EPOCHS):
    print(f'Epoch {epoch+1}/{NUM_EPOCHS}')
    print('-' * 70)

    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, scheduler, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    learning_rates.append(scheduler.get_last_lr()[0])

    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    # Print epoch summary
    print(f'\nTrain Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%')
    print(f'Learning Rate: {learning_rates[-1]:.6f}')
    print(f'Overfitting gap: {abs(train_acc - val_acc):.2f}%')

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch + 1
        patience_counter = 0

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'train_acc': train_acc,
        }, 'model.pth')

        print(f'✓ Best model saved! (Val Acc: {val_acc:.2f}%)')
    else:
        patience_counter += 1
        print(f'No improvement. Patience: {patience_counter}/{PATIENCE}')

    print(f'Best Val Acc so far: {best_val_acc:.2f}% (Epoch {best_epoch})')
    print('=' * 70 + '\n')

    # Early stopping
    if patience_counter >= PATIENCE:
        print(f'Early stopping triggered after {epoch+1} epochs!')
        print(f'Best validation accuracy: {best_val_acc:.2f}% at epoch {best_epoch}')
        break

print('\nTraining completed!')
print(f'Best validation accuracy: {best_val_acc:.2f}% at epoch {best_epoch}')

In [ ]:
# CELL 9: Plot Training History
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Loss plot
ax1.plot(train_losses, label='Train Loss', linewidth=2)
ax1.plot(val_losses, label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Accuracy plot
ax2.plot(train_accs, label='Train Acc', linewidth=2)
ax2.plot(val_accs, label='Val Acc', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy (%)', fontsize=12)
ax2.set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# Learning rate plot
ax3.plot(learning_rates, linewidth=2, color='green')
ax3.set_xlabel('Epoch', fontsize=12)
ax3.set_ylabel('Learning Rate', fontsize=12)
ax3.set_title('Learning Rate Schedule (OneCycleLR)', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.set_yscale('log')

# Overfitting gap plot
gaps = [abs(t - v) for t, v in zip(train_accs, val_accs)]
ax4.plot(gaps, linewidth=2, color='red')
ax4.set_xlabel('Epoch', fontsize=12)
ax4.set_ylabel('Overfitting Gap (%)', fontsize=12)
ax4.set_title('Train-Val Accuracy Gap', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)
ax4.axhline(y=5, color='orange', linestyle='--', label='Target (<5%)')
ax4.legend(fontsize=10)

plt.tight_layout()
plt.savefig('improved_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print(f'\nFinal Training Accuracy: {train_accs[-1]:.2f}%')
print(f'Final Validation Accuracy: {val_accs[-1]:.2f}%')
print(f'Best Validation Accuracy: {best_val_acc:.2f}%')
print(f'Final Overfitting Gap: {abs(train_accs[-1] - val_accs[-1]):.2f}%')

In [ ]:
# CELL 10: Submission Functions (Required for Instructor Testing)
# For compatibility, create an alias to the original class name
TinyImageNetCNN = ImprovedTinyImageNetCNN

def load_model(model_path='model.pth', device='cuda'):
    """Load the trained model from file"""
    model = ImprovedTinyImageNetCNN(num_classes=15, dropout_rate=0.3)

    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])

    model = model.to(device)
    model.eval()

    print(f'Model loaded successfully from {model_path}')
    print(f'Best validation accuracy during training: {checkpoint["val_acc"]:.2f}%')

    return model


def predict(model, test_loader, device='cuda'):
    """Make predictions on test data"""
    model.eval()
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    all_predictions = []

    with torch.no_grad():
        for inputs, _ in tqdm(test_loader, desc='Predicting'):
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            all_predictions.extend(predicted.cpu().numpy())

    return np.array(all_predictions)


# Test the load function
print('\nTesting load_model function...')
loaded_model = load_model('model.pth', device=device)
print('✓ Load function works correctly!')

# Verify on validation set
print('\nTesting predict function on validation set...')
predictions = predict(loaded_model, val_loader, device=device)
true_labels = val_labels
accuracy = 100. * np.sum(predictions == true_labels) / len(true_labels)

print(f'\n✓ Predict function works correctly!')
print(f'Validation Accuracy with loaded model: {accuracy:.2f}%')
print(f'Number of correct predictions: {np.sum(predictions == true_labels)}/{len(true_labels)}')

In [ ]:
# CELL 11: Final Summary
print('\n' + '='*80)
print('IMPROVED MODEL SUMMARY')
print('='*80)
print(f'Architecture: Improved Sequential CNN with Optimized Hyperparameters')
print(f'Total Parameters: {sum(p.numel() for p in loaded_model.parameters()):,}')
print(f'\nKey Improvements:')
print('  ✓ Reduced Dropout: 0.1→0.15→0.2→0.25 (conv) + 0.3 (FC)')
print('  ✓ Less Aggressive Augmentation (rotation: 10°, jitter: 0.1)')
print('  ✓ NO Label Smoothing (standard CrossEntropy)')
print('  ✓ OneCycleLR Scheduler with 30% warmup')
print('  ✓ Gradient Clipping (max_norm=1.0)')
print('  ✓ Higher Learning Rate (0.003)')
print('  ✓ Lower Weight Decay (5e-5)')
print('  ✓ Increased Patience (20 epochs)')
print(f'\nPerformance:')
print(f'  Best Validation Accuracy: {best_val_acc:.2f}%')
print(f'  Achieved at Epoch: {best_epoch}')
print(f'  Expected Improvement: +20-35% from original 44.85%')
print('='*80)

print('\n✅ READY FOR SUBMISSION!')
print('\nDownload these files from Colab:')
print('1. This notebook (.ipynb)')
print('2. model.pth')
print('\nZip them as: GroupX_Assignment.zip')